# sshについて

## SSH

離れたサーバー（コンピュータ）に安全に接続・操作する仕組み
- 暗号化されるため安全にやりとりできる
- 接続先のコンピュータでコマンド実行が可能

## SSHキー

SSHを用いて安全に通信するために使う鍵のペア  

- 秘密鍵：サーバーへの登録  

- 秘密鍵：本人確認に使用  
    - 他人には絶対に渡さない

## Gitでの使用方法

GitHubやGitLabなどのリモートリポジトリとの接続方法は２種類  
基本的にはSSH接続を使用

- HTTPS：毎回ユーザー名とパスワードを入力する必要あり  
```セキュリティ上の理由から、普通のパスワード入力は廃止```  
    - ```個人アクセストークン (PAT: Personal Access Token)```  


- SSH：鍵を登録すれば以降は自動で安全に通信できる。

## 接続の準備

### SSHキーの作成

ターミナルで作成する  
質問がいくつか出てくるが全てEnter(パスフレーズは任意で設定)  

```ssh-keygen -t ed25519 -C "メールアドレス"```  

- ```ssh-keygen```：SSHキーを作るよ
- ```-t ed25519```：鍵の種類は「ed25519」を使うよ
- ```-C "メールアドレス"```：鍵にメールのコメントを残すよ(識別用)

1. **SSHキー作成**  
```ssh-keygen -t ed25519 -C "mail-adress" -f ~/.ssh/gcp_key```  
    - ```-f ~/.ssh/gcp_key```：ファイル名を指定  
        - GCP(Google Cloud Platform)上ではメールアドレスではなく、任意のユーザー名を使用する。

### 公開鍵の中身をコピー

```cat ~/.ssh/id_ed25519.pub```  

表示された文字列（公開鍵）を全部コピーする。  

- ```cat```：ファイルの内容を表示するLinuxコマンド
- ```~/.ssh/id_ed25519.pub```：.ssh/の中に保存されているid_ed25519.pub(公開鍵)までのパス  

※~/.sshでホームディレクトリ内の.sshフォルダを指し、この位置に今まで作ったsshキーのデータが保存されている

## リモートの設定

### GitHub

1. ブラウザでGitHubにログイン  

2. プロフィールからSettingsを開く  

3. 「SSH and GPG keys」を開き、「New SSH key」をクリック  

4. 先ほどコピーした公開鍵を貼り付ける  

5. タイトルを適当なものにして「Add SSH key」をクリック

### GitLab

1. ブラウザでGitLabにログイン  

2. プロフィールから「設定」か「プロファイルの編集」を開く  

3. 「SSHキー」を開き、「新しいキーを追加」をクリック  

4. 先ほどコピーした公開鍵を貼り付ける  

5. タイトルを適当なものにして「キーを追加」をクリック

## ローカルへClone

リモート(GitHub/GitLab)のリポジトリをローカル環境にコピー  
ローカルのフォルダ構造は、下記のようにworkspace配下にリポジトリを配置想定

📂 workspace  
├── 📁 projectA  
├── 📁 projectB  
├── 📁 data  
⋮  
└── 📁 test-git ← リモートからcloneして作られるフォルダ

1. リモートで、接続したいリポジトリを開く  

2. 「Code」をクリックして、SSHタブからURLをコピー  
```10/20251008053019894.png```  

3. ローカルでターミナルを開き、workspaceディレクトリへ移動  
`cd 〇〇/△△/workspace/`  

4. 次のコマンドを実行  
`git clone コピーしたURL`  

6. リポジトリのフォルダへ移動  
`cd test-git`

## 確認

`cd ~/.ssh`   
sshキーが保存されているフォルダに移動  

`ls`          
#フォルダの中身を確認

例）  
config              
鍵やホストの設定をまとめるファイル  

id_ed25519         
秘密鍵(中身は内緒)  

id_ed25519.pub      
公開鍵(リモートに登録)  

id_rsa              
秘密鍵(中身は内緒)  

id_rsa.pub         
公開鍵(リモートに登録)  

known_hosts         
接続履歴(再接続時の確認を省略)  

known_hosts.old     
過去の接続履歴  

<n/><n/>
GitHubとGitLabのどちらも登録しているため、SSHキーが２種類あります。  
SSHキーをたくさん作ってもconfigに整理しておけば見返すことが可能です。

## config　演習

1. 接続に必要な情報
    - ipアドレス　ex）4.121.246.213  
    - ユーザー名　ex）任意（現場先から指定があればそれに従う）  

    - 鍵
        1. 鍵なしで接続  
        パスワードが必要  

        2. 鍵ありで接続
            - 現場に鍵ファイルあり  
            鍵ファイルをwsl上の.sshフォルダに入れる  
            - 鍵ファイルなし  
            .sshフォルダに鍵ファイルを作成する。作成方法は上記に記載   

2. 対象サーバーまたはブラウザ操作  
鍵の情報を伝えるため、.pubファイルに記載されているコードをすべて張り付ける。 

3. リモートエクスプローラー操作  
SSHで新しいリモートを作成（＋）を選択し、鍵の場所を伝えてログインするsshコマンドをうつ。
ex）```ssh kouty@34.121.246.213 -i ~/.ssh/gcp_key```  
    - 注意点  
    configファイルのIdentityFileに記載されている```~```は、wsl上ではなくPCのホームディレクトリを示す。現状、そこに合わせるために鍵ファイルをPCの.ssh/に入れる必要あり。（configファイルを開く方法は後述）

4. configの書き換え  
リモートエクスプローラー上でSSH欄の構成ファイル設定を選択  
Hostの内容は任意の名前に書き換えることで容易にwsl上でサーバーを呼び出すことができる。  
```ssh 任意の名前```  
    - 各項目の意味  
    ```Host```：接続時に使う任意の名前を指定
    ```HostName```：接続先サーバーのIPアドレスまたはドメイン名　ex）34.121.246.213  
    ```User```：ログインする時のユーザー名　ex）kouty  
    ```IdentityFile```：認証に使用する秘密鍵の保存先を示す　ex）~/.ssh/gcp_key

**その他**  
- ```ssh -T git@github.com```  
GitHubとのSSH接続確認

- ```df -h```：ディスクの空き容量を表示する。カレントディレクトリはhomeで行う。  
disk free -human readable  
特に見るべきは Mounted on が /（ルート）になっている行  
OSやプログラムがインストールされるメインの場所。ここのAvailが少ないと、新しいソフトをインストールしたり、Pythonのライブラリをダウンロードしたりする際にエラー（No space left on device）が発生。  
ssh接続もしずらくなる。  

- ```top```  (cpuの確認)  
実験などで使用している人がいるか確認

- ```nvidia-smi```  （gpuの確認）  
colabで確認できる。実験している人の配慮できる。  
重いとサーバーが落ちる可能性がある。自身の実験で何パーセント使用するかを配慮するなど行う。

- エラー確認　以下が表示されたら別でターミナルを開いたら解消されるかも。codeコマンドを入力した際に起こり得る
```py
Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1000/vscode-ipc-d68ab422-9241-49f9-b110-b9f91995db3e.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1637:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1000/vscode-ipc-d68ab422-9241-49f9-b110-b9f91995db3e.sock'
}
```
